In [1]:
import pandas as pd
import json
import requests

In [2]:
df_sales = pd.read_csv( '../../data/test.csv', low_memory=False )
df_store = pd.read_csv( '../../data/store.csv', low_memory=False )

# merge
df_test = pd.merge( df_sales, df_store, how='left', on='Store' )

In [3]:
# choose store for prediction
df_test = df_test[df_test['Store'].isin( [20, 23, 22, 24] )]
# remove closed days
df_test = df_test[df_test['Open'] != 0]
df_test = df_test[~df_test['Open'].isnull()]
df_test = df_test.drop( 'Id', axis=1 )

In [4]:
# convert Dataframe to json
data = json.dumps( df_test.to_dict( orient='records' ) )

In [9]:
# API Call
url = 'https://web-production-bdd61.up.railway.app/rossmann/predict'
header = {'Content-type': 'application/json' } 
data = data

r = requests.post( url, data=data, headers=header )
print( 'Status Code {}'.format( r.status_code ) )

Status Code 500


In [6]:
d1 = pd.DataFrame( r.json(), columns=r.json()[0].keys() )

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
d2 = d1[['store', 'prediction']].groupby( 'store' ).sum().reset_index()

for i in range( len( d2 ) ):
    print( 'Store Number {} will sell R${:,.2f} in the next 6 weeks'.format( 
            d2.loc[i, 'store'], 
            d2.loc[i, 'prediction'] ) )

Store Number 20 will sell R$213,185.51 in the next 6 weeks
Store Number 22 will sell R$130,922.42 in the next 6 weeks
Store Number 23 will sell R$245,222.39 in the next 6 weeks
Store Number 24 will sell R$371,500.51 in the next 6 weeks
